* After imputation always check the following things :-<br>
-> The variance it get sprase , if it get supress too much then it is "Red Flag" eg:- 206 se 161.<br>
-> The shape/ distribution of curve, it shoud not be change too much.<br>
-> The correlation of imputed column with other columns.

In [1]:
import pandas as pd
import seaborn as sns
import scipy.stats as stats
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split,GridSearchCV,cross_val_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler,OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier

In [2]:
df = pd.read_csv("titanic.csv")
df["Family"]=df["SibSp"] + df["Parch"]
df.drop(["PassengerId","Name","Cabin","Ticket","SibSp","Parch"],axis=1,inplace=True)
df

,Survived,Pclass,Sex,Age,Fare,Embarked,Family
0,0,3,male,22.0,7.2500,S,1
1,1,1,female,38.0,71.2833,C,1
2,1,3,female,26.0,7.9250,S,0
3,1,1,female,35.0,53.1000,S,1
4,0,3,male,35.0,8.0500,S,0
...,...,...,...,...,...,...,...
886,0,2,male,27.0,13.0000,S,0
887,1,1,female,19.0,30.0000,S,0
888,0,3,female,NaN,23.4500,S,3
889,1,1,male,26.0,30.0000,C,0


In [ ]:
x_train,x_test,y_train,y_test = train_test_split(df.drop(["Survived"],axis=1),df["Survived"],test_size=0.2,random_state=1)
num = ["Pclass","Fare","Family","Age"]
cat = ["Sex","Embarked"]



num_tr = Pipeline(
    steps=[
        ("imp",SimpleImputer(strategy="median")),
        ("scl",StandardScaler(with_mean=False))
    ]
)
cat_tr = Pipeline(
    steps=[
        ("imp",SimpleImputer(strategy='most_frequent')),
        ("ohe",OneHotEncoder(handle_unknown='ignore')),
        ("scl",StandardScaler(with_mean=False))
    ]
)


preprocessor = ColumnTransformer(
    transformers=[
        ("num_tr",num_tr,num),
        ("cat_tr",cat_tr,cat)
    ]
)

clf = Pipeline(
    steps=[
        ("pre",preprocessor),
        ("model",RandomForestClassifier(max_depth=10,n_estimators=50))
    ]
)

clf.fit(x_train,y_train)
print(accuracy_score(y_test,clf.predict(x_test))*100)

cross_score = cross_val_score(clf,x_test,y_test,cv=10).mean()*100
print(cross_score)

# param_dict = {
#     "pre__num_tr__imp__strategy":["mean","median","constant"],
#     "pre__cat_tr__imp__strategy":["most_frequent","constant"],
#     "pre__num_tr__imp__fill_value":[None,999],
#     "pre__cat_tr__imp__fill_value":[None,"missing"],
#     "pre__num_tr__imp__add_indicator":[True,False],
#     "model__n_estimators": [50, 100, 200],
#     "model__max_depth": [None, 10, 20, 30],
 
# }

# grid_clf = GridSearchCV(clf,param_dict,cv=10)
# grid_clf.fit(x_train,y_train)
# print("Best Parameters:", grid_clf.best_params_)
# print("Best Score:", grid_clf.best_score_)


78.77094972067039
